In [2]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import io

In [29]:
widgets.Label().style.keys

['_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'background',
 'description_width',
 'font_family',
 'font_size',
 'font_style',
 'font_variant',
 'font_weight',
 'text_color',
 'text_decoration']

In [6]:
bigbutton = widgets.Layout(width="290px", height="60px", margin="5px 5px 5px 5px")
actionbutton = widgets.Layout(width="140px", height="60px", margin="5px 5px 5px 5px")
value_label = widgets.Layout(width="60px", height="60px", margin="20px 20px 20px 20px")
padding10 = widgets.Layout(margin="10px 10px 10px 10px")

In [ ]:
class Visualization:
    def __init__(self):
        self.setup()
        with self.output:
            self.game = Game(self)

    def setup(self):
        self.deal_button = widgets.Button(description="deal", layout=bigbutton)
        self.deal_button.on_click(self.deal)

        self.reset_button = widgets.Button(description="reset", layout=bigbutton, disabled=True)
        self.reset_button.on_click(self.reset)

        self.deck_info_label = widgets.HTML(value="No deck info available", layout=widgets.Layout(object_position="right", width="250px"))
        self.deck_box = widgets.VBox([self.deck_info_label], layout=padding10)

        self.payout_graph = widgets.Image(value=self.get_payout_graph(), format="png", width=800, height=600)

        self.dealer_image = widgets.Image(
            value=open("images/dealer.png", "rb").read(), 
            format="png",
            height=120,
            width=120
        )
        self.dealer_value_label = widgets.Label(value="-", style={"font_size": "300%"}, layout=value_label)
        self.dealer_cards_label = widgets.Label(value="[]", style={"font_size": "150%"})
        self.dealer_box = widgets.HBox([self.dealer_image, self.dealer_value_label, self.dealer_cards_label])

        self.player_image = widgets.Image(
            value=open("images/player.png","rb").read(), 
            format="png",
            height=120,
            width=120
        )
        self.player_value_label = widgets.Label(value="-", style={"font_size": "300%"}, layout=value_label)
        self.player_cards_label = widgets.Label(value="[]", style={"font_size": "150%"})
        self.player_box = widgets.HBox([self.player_image, self.player_value_label, self.player_cards_label])


        self.hit_button = widgets.Button(description="hit", layout=actionbutton, disabled=True)
        self.stand_button = widgets.Button(description="stand", layout=actionbutton, disabled=True)
        self.double_button = widgets.Button(description="double", layout=actionbutton, disabled=True)
        self.split_button = widgets.Button(description="split", layout=actionbutton, disabled=True)

        self.hit_bust_chance = widgets.FloatProgress(min=0.0, max=1.0)
        self.hit_bust_chance.layout.width = "590px"
        self.hit_bust_chance.margin = "5px 5px 5px 5px"
        self.hit_bust_chance.style.description_width = "150px"
        self.set_hit_bust_chance(0.0)

        self.stand_win_chance = widgets.FloatProgress(min=0.0, max=1.0)
        self.stand_win_chance.layout.width = "590px"
        self.stand_win_chance.margin = "5px 5px 5px 5px"
        self.stand_win_chance.style.description_width = "150px"
        self.set_stand_win_chance(0.0)

        self.blackjack_chance = widgets.FloatProgress(min=0.0, max=1.0)
        self.blackjack_chance.layout.width = "590px"
        self.blackjack_chance.margin = "5px 5px 5px 5px"
        self.blackjack_chance.style.description_width = "150px"
        self.set_blackjack_chance(0.0)

        
        self.output = widgets.Output()

        self.menu_box = widgets.HBox([self.deal_button, self.reset_button], layout=padding10)

        self.action_box = widgets.HBox([self.hit_button, self.stand_button, self.double_button, self.split_button], layout=padding10)

        self.chance_box = widgets.VBox([self.hit_bust_chance, self.stand_win_chance, self.blackjack_chance], layout=padding10)

        self.game_box = widgets.VBox([self.menu_box, self.action_box, self.chance_box, self.dealer_box, self.player_box])

        self.upper_box = widgets.HBox([self.game_box, self.deck_box, self.payout_graph])

        display(self.upper_box, self.output)

    def deal(self, b):
        self.reset_cards()
        self.reset_button.disabled = False
        self.deal_button.disabled = True
        with self.output:
            clear_output()
            self.game.play_round()
  
    def reset(self, b):
        with self.output:
            clear_output()
            print("anognfobgfodibiadfoga")
            self.game = Game(self)
            print("game payout", self.game.payouts)
            
        self.update_payout_graph([0])
        self.reset_cards()
        self.reset_button.disabled = True
        self.deal_button.disabled = False
        self.reset_colors()
        self.disable_action_buttons()

    def reset_cards(self):
        self.dealer_cards_label.value = "[]"
        self.player_cards_label.value = "[]"
        self.dealer_value_label.value = "-"
        self.player_value_label.value = "-"
        
    
    def disable_action_buttons(self):
        self.hit_button.disabled = True
        self.stand_button.disabled = True
        self.double_button.disabled = True
        self.split_button.disabled = True
    
    
    def reset_colors(self):
        self.dealer_value_label.style.text_color = 'black'
        self.player_value_label.style.text_color = 'black'
        self.dealer_value_label.style.font_weight = "normal"
        self.player_value_label.style.font_weight = "normal"
        self.dealer_cards_label.style.text_color = 'black'
        self.player_cards_label.style.text_color = 'black'
        self.dealer_cards_label.style.font_weight = "normal"
        self.player_cards_label.style.font_weight = "normal"
        self.activation(0,0)
    
    def set_hit_bust_chance(self, value):
        self.hit_bust_chance.value = value
        self.hit_bust_chance.description = f"Hit bust chance {value:.2f}"
        self.hit_bust_chance.bar_style = "danger" if value > 0.5 else "success"
    
    def set_stand_win_chance(self, value):
        self.stand_win_chance.value = value
        self.stand_win_chance.description = f"Stand Win chance {value:.2f}"
        self.stand_win_chance.bar_style = "danger" if value < 0.5 else "success"
    
    def set_blackjack_chance(self, value):
        self.blackjack_chance.value = value
        self.blackjack_chance.description = f"Blackjack chance {value:.2f}"
        self.blackjack_chance.bar_color = "gold"

    def update_deck_info(self, deck):
        deck_size = len(deck)
        deck_info = deck.get_value_counts()
        html = """
        <h2>DECK INFO</h2>
            <p style="font-size:20px">
                <table>
                    <tr>
                        <th>Value</th>
                        <th>Occurence</th>
                        <th>Percentage</th>
                    </tr>"""
        for card, count in deck_info.items():
            html += f"""<tr>
                        <td>{card}</td>
                        <td>{count}</td>
                        <td>{100*count/deck_info.values.sum():.2f}</td>
                    </tr>"""
        html += """
                </table>
            </p>""" 
        self.deck_info_label.value = html


    def update_payout_graph(self, payouts):
        cumsum_payouts = np.cumsum(np.array(payouts))
        self.payout_graph.value = self.get_payout_graph(cumsum_payouts)

    def get_payout_graph(self, cumsum_payouts=np.zeros(1)):
        positive = cumsum_payouts >= 0
        negative = cumsum_payouts <= 0
        
        plt.ioff()
        plt.clf()
        plt.style.use('ggplot')
        plt.title("cumulative payout over time")
        plt.xlabel("rounds")
        plt.ylabel("cumulative payout")
        plt.plot(np.arange(len(cumsum_payouts))[positive], cumsum_payouts[positive], color="green")
        plt.fill_between(np.arange(len(cumsum_payouts))[positive], cumsum_payouts[positive], color="green", alpha=0.5)

        plt.plot(np.arange(len(cumsum_payouts))[negative], cumsum_payouts[negative], color="red")
        plt.fill_between(np.arange(len(cumsum_payouts))[negative], cumsum_payouts[negative], color="red", alpha=0.5)

        plt.xlim(plt.xlim()[0], plt.xlim()[1]+1)

        plt.axhline(y=cumsum_payouts[-1], color='gray', alpha=0.5)

        yabs_max = abs(max(plt.ylim(), key=abs))
        plt.ylim(ymin=-yabs_max, ymax=yabs_max)  

        buf = io.BytesIO()  
        plt.savefig(buf, format='png')  
        buf.seek(0) 
        returnval = buf.getvalue()
        del buf 
        return returnval

    def activation(self, dealer, player):
        if dealer == 1:
            self.dealer_image.value = open("images/dealer_active.png", "rb").read()
        else:
            self.dealer_image.value = open("images/dealer.png", "rb").read()
        
        if player == 1:
            self.player_image.value = open("images/player_active.png", "rb").read()
        else:
            self.player_image.value = open("images/player.png", "rb").read()
